In [1]:
!pip install -q youtube-transcript-api langchain-community faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatibl

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

Step 1a : Indexing (Document Ingestion)

In [3]:
video_id = "LPZh9BOjkQs"
try:
  ytt_api = YouTubeTranscriptApi()
  fetched_transcript = ytt_api.fetch(video_id)
  transcript = " ".join(snippet.text for snippet in fetched_transcript)
except:
  print("No transcript available for this video")

print(transcript)

Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the use

Step 1b : Text Splitting

In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
chunk = splitter.create_documents([transcript])
print(len(chunk))
print(chunk[0].page_content)

10
Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the


Step 1c : Generating Embeddings

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name = 'BAAI/bge-m3',
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings' : True}
)
vector_store = FAISS.from_documents(chunk,embeddings)

/tmp/ipykernel_36/2777707566.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-08-20 13:26:57.560836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755696417.738336      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755696417.790858      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alrea

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
vector_store.index_to_docstore_id

{0: '028f996e-1ab9-4c38-8968-18c2e4e8431a',
 1: '32c54a4d-d704-4f89-832d-932f6aa7944c',
 2: '4e1b6b86-0f8e-47b0-9931-226dff88bda1',
 3: 'ef77c6c9-d163-4bb8-ae51-e63e9ca4d12e',
 4: '5ab13fbd-20df-473d-bcf2-f2038ce08010',
 5: '7daa3baf-4edb-4889-8d11-d0c7a894ac76',
 6: '8e565de0-17a7-45b8-ac25-dd200b9f825e',
 7: '42661f73-bc3d-4ba2-8196-dc178e2d19da',
 8: '3030ee90-9d54-4007-a245-db789d32e75c',
 9: '4b6dd450-ffb1-4ab5-85f6-e1067e8b5ab3'}

Step 2 : Retrieval

In [36]:
retriever = vector_store.as_retriever(search_type = 'similarity',search_kwargs = {'k': 3})
retriever.invoke("What is deepmind ?")

[Document(id='7daa3baf-4edb-4889-8d11-d0c7a894ac76', metadata={}, page_content="called reinforcement learning with human feedback. Workers flag unhelpful or problematic predictions, and their corrections further change the model's parameters, making them more likely to give predictions that users prefer. Looking back at the pre-training, though, this staggering amount of computation is only made possible by using special computer chips that are optimized for running many operations in parallel, known as GPUs. However, not all language models can be easily parallelized. Prior to 2017, most language models would process text one word at a time, but then a team of researchers at Google introduced a new model known as the transformer. Transformers don't read text from the start to the finish, they soak it all in at once, in parallel. The very first step inside a transformer, and most other language models for that matter, is to associate each word with a long list of numbers. The reason fo

Step 3 : Augmentation

In [8]:
!pip install -q langchain-huggingface transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 13.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.


In [13]:
import getpass
import os

os.environ["HF_TOKEN"] = getpass.getpass(
    "Enter your Hugging Face API key:"
)

Enter your Hugging Face API key: ········


In [37]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=True,
    provider="auto",  
)

chat_model = ChatHuggingFace(llm=llm)
print(chat_model.invoke("Tell me a joke"))

content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 39, 'total_tokens': 93}, 'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None} id='run--56a1f3eb-bcc6-4e58-9b3c-9d7798016272-0' usage_metadata={'input_tokens': 39, 'output_tokens': 54, 'total_tokens': 93}


In [15]:
prompt = PromptTemplate(
    template = """
    You are a helpful assistant.
    Answer only from the provided transcript context.
    If context is insufficient just say you dont know.

    {context}
    Question : {question}
    """,
    input_variables = ['context','question']
)

In [38]:
question = 'Was the topic of backpropagation discussed in this video ? If yes then what was discussed'
retrieved_docs = retriever.invoke(question)

In [39]:
retrieved_docs

[Document(id='4b6dd450-ffb1-4ab5-85f6-e1067e8b5ab3', metadata={}, page_content='the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.'),
 Document(id='ef77c6c9-d163-4bb8-ae51-e63e9ca4d12e', metadata={}, page_content="on many example pieces of text. One of these training examples could be just a handful of words, or it could be thousands, but in either case, the way this works is to pass in all but the last word from that example into the model and compare the prediction that it makes with the true last word from the example. An algorithm called backpropagation is used to tweak all of the parameters in such a way that it makes the model a little more likely to choose the true last word and a little less likely to choo

In [40]:
context_text = '\n\n'.join(doc.page_content for doc in retrieved_docs)
context_text

"the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\non many example pieces of text. One of these training examples could be just a handful of words, or it could be thousands, but in either case, the way this works is to pass in all but the last word from that example into the model and compare the prediction that it makes with the true last word from the example. An algorithm called backpropagation is used to tweak all of the parameters in such a way that it makes the model a little more likely to choose the true last word and a little less likely to choose all the others. When you do this for many, many trillions of examples, not only does the model start to give more accurate predictions on the training data,

In [41]:
final_prompt = prompt.invoke({'context':context_text,'question':question})
final_prompt

StringPromptValue(text="\n    You are a helpful assistant.\n    Answer only from the provided transcript context.\n    If context is insufficient just say you dont know.\n\n    the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\non many example pieces of text. One of these training examples could be just a handful of words, or it could be thousands, but in either case, the way this works is to pass in all but the last word from that example into the model and compare the prediction that it makes with the true last word from the example. An algorithm called backpropagation is used to tweak all of the parameters in such a way that it makes the model a little more likely to choose the true last word and a little le

In [42]:
ans = chat_model.invoke(final_prompt)
print(ans.content)

Yes, the topic of backpropagation was discussed in this video. According to the transcript, backpropagation is an algorithm used to tweak all of the parameters in the model in such a way that it makes the model a little more likely to choose the true last word and a little less likely to choose all the others. It is used to train the model by performing comparisons between the model's predictions and the true last word from the example.


In [43]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [44]:
def format_docs(retrieved_docs):
    context_text = '\n\n'.join(doc.page_content for doc in retrieved_docs)
    return context_text

In [47]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [48]:
parallel_chain.invoke("Who is Andrew Ng ?")

{'context': "the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\nnext word. Although researchers design the framework for how each of these steps work, it's important to understand that the specific behavior is an emergent phenomenon based on how those hundreds of billions of parameters are tuned during training. This makes it incredibly challenging to determine why the model makes the exact predictions that it does. What you can see is that when you use large language model predictions to autocomplete a prompt, the words that it generates are uncannily fluent, fascinating, and even useful. If you're a new viewer and you're curious about more details on how transformers and attention work, boy do I have some mat

In [33]:
parser = StrOutputParser()

In [49]:
main_chain = parallel_chain | prompt | chat_model | parser
main_chain.invoke("Can you summarize the video ?")

"The video discusses the transformer model and its components. It explains that the transformer model doesn't process text one word at a time, but rather soaks it in all at once in parallel. The video also touches upon the concept of reinforcement learning with human feedback, where workers correct unhelpful or problematic predictions to improve the model's parameters.\n\nThe video mentions that the training process of the transformer model works with continuous values, which is achieved by associating each word with a long list of numbers. However, the video also notes that the specific behavior of the model is an emergent phenomenon based on the tuning of its parameters during training, making it challenging to determine why the model makes specific predictions.\n\nThe video also mentions the importance of the attention mechanism in the transformer model and provides links to additional content, including a series on deep learning and a talk given at the company TNG in Munich."